# 🎨 SD-DarkMaster-Pro Unified Interface

This is a single-cell notebook that launches a comprehensive Streamlit interface for managing all SD WebUI operations.

**Features:**
- ✅ One-click setup and configuration
- 📦 Model browsing and selection (SD1.5, SDXL, CivitAI)
- 💾 Download management with progress tracking
- 🚀 WebUI launcher with multiple options
- 🧹 Storage management and cleanup
- 📊 Real-time monitoring and logs

Simply run the cell below to access the complete interface!

In [ ]:
#@title 🚀 Launch SD-DarkMaster-Pro Unified Interface
#@markdown This single cell handles everything - setup, downloads, and WebUI launch!

import os
import sys
import subprocess
import time
from pathlib import Path

# Configuration
ngrok_token = "" #@param {type:"string"}
tunnel_choice = "cloudflared" #@param ["cloudflared", "ngrok", "localtunnel", "none"]
auto_launch = True #@param {type:"boolean"}

print("🎨 SD-DarkMaster-Pro Unified Interface")
print("=" * 50)

# Detect platform
if os.path.exists('/content'):
    platform = 'colab'
    base_path = Path('/content')
elif os.path.exists('/kaggle'):
    platform = 'kaggle'
    base_path = Path('/kaggle/working')
else:
    platform = 'local'
    base_path = Path.home()

print(f"🖥️  Platform: {platform}")
project_path = base_path / 'SD-DarkMaster-Pro'

# Clone repository if needed
if not project_path.exists():
    print("\n📥 Cloning repository...")
    !git clone https://github.com/remphanostar/SD-DarkMaster-Pro.git {project_path}
else:
    print("\n📂 Repository already exists")
    os.chdir(project_path)
    !git pull

# Install requirements
print("\n📦 Installing requirements...")
requirements = ['streamlit', 'pyngrok', 'psutil', 'requests', 'beautifulsoup4']
for req in requirements:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', req])

# Install cloudflared if needed
if tunnel_choice == 'cloudflared' and not os.path.exists('/usr/local/bin/cloudflared'):
    print("\n📥 Installing Cloudflare Tunnel...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
    !chmod +x cloudflared
    !sudo mv cloudflared /usr/local/bin/

# Set ngrok token
if ngrok_token:
    os.environ['NGROK_AUTH_TOKEN'] = ngrok_token

# Launch Streamlit app
print("\n🚀 Launching unified interface...")
print("-" * 50)

# Kill any existing Streamlit processes
subprocess.run(['pkill', '-f', 'streamlit'], capture_output=True)
time.sleep(2)

# Start Streamlit in background
app_path = project_path / 'scripts' / 'unified_app.py'
process = subprocess.Popen([
    sys.executable, '-m', 'streamlit', 'run',
    str(app_path),
    '--server.port', '8501',
    '--server.headless', 'true',
    '--logger.level', 'error'
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Wait for server to start
print("⏳ Starting Streamlit server...")
time.sleep(5)

# Create tunnel based on choice
public_url = None

if tunnel_choice == "cloudflared":
    print("\n🌐 Creating Cloudflare Tunnel...")
    # Start cloudflared in background
    cf_process = subprocess.Popen(
        ['cloudflared', 'tunnel', '--url', 'http://localhost:8501'],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    
    # Wait for URL
    for i in range(30):
        line = cf_process.stderr.readline()
        if 'trycloudflare.com' in line:
            public_url = line.split('https://')[1].split(' ')[0]
            public_url = f'https://{public_url}'
            break
        time.sleep(0.5)
    
elif tunnel_choice == "ngrok" and ngrok_token:
    try:
        from pyngrok import ngrok
        # Set auth token
        ngrok.set_auth_token(ngrok_token)
        # Close any existing tunnels
        for tunnel in ngrok.get_tunnels():
            ngrok.disconnect(tunnel.public_url)
        # Create new tunnel
        public_url = ngrok.connect(8501)
    except Exception as e:
        print(f"\n⚠️  Ngrok error: {e}")
        
elif tunnel_choice == "localtunnel":
    print("\n🌐 Creating LocalTunnel...")
    !npm install -g localtunnel 2>/dev/null
    lt_process = subprocess.Popen(
        ['lt', '--port', '8501'],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    # Wait for URL
    for i in range(10):
        line = lt_process.stdout.readline()
        if 'your url is:' in line.lower():
            public_url = line.split('is:')[1].strip()
            break
        time.sleep(0.5)

# Display access information
if public_url:
    print(f"\n✅ Interface is ready!")
    print(f"\n🌐 Access your dashboard at: {public_url}")
    print(f"\n📱 Share this link to access from any device")
else:
    print(f"\n✅ Interface is running locally!")
    print(f"\n💡 Access at: http://localhost:8501")
    if platform == 'colab':
        print(f"\n⚠️  For Colab, you need a tunnel. Try setting tunnel_choice to 'cloudflared'")

print("-" * 50)

# Display instructions
print("\n📖 Quick Start Guide:")
print("1. Click the link above to open the interface")
print("2. Navigate to 'Setup' to configure your environment")
print("3. Go to 'Models' to select what to download")
print("4. Use 'Downloads' to manage your download queue")
print("5. Launch your preferred WebUI from 'Launch'")
print("6. Monitor everything from the 'Home' dashboard")

# Keep the cell running
print("\n✨ Interface is running. This cell will stay active.")
print("⚠️  Don't close this cell or the interface will stop!")

# Optional: Auto-open the URL
if auto_launch and 'public_url' in locals():
    try:
        from IPython.display import Javascript
        display(Javascript(f'window.open(\"{public_url}\", \"_blank\");'))
    except:
        pass

# Keep process alive
try:
    while True:
        time.sleep(1)
        if process.poll() is not None:
            print("\n❌ Streamlit process ended unexpectedly")
            break
except KeyboardInterrupt:
    print("\n👋 Shutting down...")
    process.terminate()